In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accuracy _ 95]/fomat-of-gst-tax-invoice._std.jpg
/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accuracy _ 95]/invis1._std.jpg
/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accuracy _ 95]/New_Bill._std.jpg
/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accuracy _ 95]/FuelPump_Bill._std.jpg
/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accuracy _ 95]/simplified-tax-invoice-2._std.jpg
/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accuracy _ 95]/aishka_may_month_vendor_invoice-2_std.jpg
/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accuracy _ 95]/Perfect-Design-of-GST-Invoice-Format-in-Excel-Download-xlsx-file_std.jpg
/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accuracy _ 95]/003-Sales-Order-2-Deposit-SST_1._std.jpg
/kaggle/input/unscript/Final_Hackathon_Data (1)/very_good_images [Accur

In [4]:
!pwd
!cp -r ../input/unscript/'Final_Hackathon_Data (1)' ../working/



/kaggle/working


In [5]:
!cd 'Final_Hackathon_Data (1)'
!pwd

/kaggle/working


In [6]:
!pip install keras_tuner

In [19]:
import tensorflow as tf
import numpy as np
import os
from tensorflow import keras
import keras_tuner as kt
import cv2
from sklearn.model_selection import train_test_split
IMG_WIDTH = 256
IMG_HEIGHT = 256
brightness = []
contrast = []
file_names = []
img_type = []
blurs = []
# Define the model architecture
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Lambda(lambda x: tf.image.rgb_to_grayscale(x)))
    model.add(tf.keras.layers.Lambda(lambda x: tf.image.resize(x, (256, 256))))
    model.add(tf.keras.layers.Lambda(lambda x: tf.keras.utils.normalize(x, axis=-1)))
    for i in range(hp.Int('num_layers', 2, 4)):
        model.add(tf.keras.layers.Conv2D(
            filters=hp.Int(f'conv_{i}_filters', 32, 64, step=32),
            kernel_size=(5, 5),
            activation=hp.Choice(f'conv_{i}_activation', ['relu', 'tanh'])
        ))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Int('dense_units', 32, 128, step=32),
        activation=hp.Choice('dense_activation', ['relu', 'tanh'])
    ))
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load the dataset
def load_dataset(path, labels):
    images = []
    targets = []
    for label, sub_folder in zip(labels, os.listdir(path)):
        sub_path = os.path.join(path, sub_folder)
        for file_name in os.listdir(sub_path):
            file_names.append(file_name)
            img_type.append(sub_folder)
            file_path = os.path.join(sub_path, file_name)
            image = cv2.imread(file_path)
            image = cv2.resize(image,(256,256))
            im = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            brightness.append(np.mean(im))
            contrast.append(np.std(im))
            blurs.append(np.std(np.abs(cv2.Laplacian(im,cv2.CV_64F))))
            images.append(image)
            targets.append(label)
    return images, targets

x, y = load_dataset('Final_Hackathon_Data (1)', [0, 1, 2, 3])


# Split the data into training, validation, and testing sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5)

# # Initialize the tuner
# tuner = kt.RandomSearch(
#     build_model,
#     objective='val_accuracy',
#     max_trials=5,
#     directory='my_dir'
# )


# tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))
# best_model = tuner.get_best_models()[0]

# print(brightness)
# print(contrast)
import pandas as pd
data = {"images":file_names,"brightness":brightness,"contrast":contrast,"blur":blurs,"Type":img_type}
df = pd.DataFrame(data)
df.to_csv("df.csv", index=False)
print(df)

                              images  brightness   contrast        blur  \
0                 0000137486_std.jpg  240.992371  49.871314   78.422558   
1               PMI-472131-1_std.jpg  195.847382  72.520783   74.269354   
2               PMI-475376-1_std.jpg   84.686691  74.239760   74.571516   
3                     image._std.jpg  231.429489  57.306739  117.905739   
4             maxresdefault._std.jpg  227.246704  59.157562  106.894105   
..                               ...         ...        ...         ...   
65                 New_Bill._std.jpg  245.268875  33.150282   88.827440   
66  Sales_Invoice__19-20-001_std.jpg  230.775406  58.781983  148.401410   
67       GST-invoice-sample._std.jpg  239.964737  45.456260  118.271176   
68                    Book3._std.jpg  218.959503  69.904169  155.368312   
69              tax-invoice._std.jpg  240.103699  37.626909   85.528524   

                                Type  
0   Good_Images [85 _ Accuracy _ 95]  
1   Good_Images [85 _

ValueError: unknown url type: '/'